In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [1]:
# Load the parameters
%run ./Covid-19_prob_parameters.ipynb


In [8]:
def severe_prob_update(x, a_hosp_bed, state_num, n_age_group, young_age_first):
    # Severe case update testing
    Psh = np.zeros((n_age_group))
#     print("a_hosp_bed: ", a_hosp_bed)
    
    if a_hosp_bed > 0:
        n_severe = x[:, state_num["s_state"]]
        zero_mask = n_severe == 0
        
        if not young_age_first:
            n_severe = np.array(x.sum(axis=0)[state_num["s_state"]])
            
        

#         a_hosp_bed = t_hosp_bed - x.sum(axis=0)[state_num["h_state"]] # available hospital beds = total - number of patients in hospital beds


        severe_mat_acc = n_severe.cumsum(axis=0)
        hosp_bed_left = a_hosp_bed - np.insert(severe_mat_acc, 0, 0)[:-1]


        if len(severe_mat_acc) == 1:
            severe_mat_acc = np.ones(n_age_group)*severe_mat_acc[0]
            hosp_bed_left = np.ones(n_age_group)*hosp_bed_left[0]
                        
        

        print("n_severe: ", n_severe)
#         print("severe_mat_acc: ", severe_mat_acc)
#         print("a_hosp_bed: ", a_hosp_bed)
        print("hosp_bed_left: ", hosp_bed_left)

        full_fill_mask = np.bitwise_and((severe_mat_acc <= a_hosp_bed), ~zero_mask)

#         print("full_fill_mask: ", full_fill_mask)

        partial_fill_mask = np.bitwise_and(np.bitwise_and(~full_fill_mask, hosp_bed_left >= 0), ~zero_mask)

#         print("partial_fill_mask: ", partial_fill_mask)

        if full_fill_mask.any():
            Psh[full_fill_mask] = 1
        if partial_fill_mask.any():
            Psh[partial_fill_mask] = \
            np.array(hosp_bed_left[partial_fill_mask]/n_severe[partial_fill_mask]).flatten()

#         if young_age_first:
#             a_hosp_bed = a_hosp_bed - np.dot(Psh, n_severe)
#             a_hosp_bed = a_hosp_bed[0]
#         else:
#             a_hosp_bed = a_hosp_bed - Psh[0,0]*n_severe
            
        print("Psh: ", Psh)
    
    return Psh



In [27]:
# Testing severe_prob_update, remeber to comment after testing, otherwise would throw error in loading function in other notebook
# young_age_first = True
# t_hosp_bed = 500

# n_age_group = 9

# test_array = np.array([10, 20, 25, 30, 50, 80, 100, 150, 300])
# x = np.repeat(test_array.reshape(9, 1), 10, axis=1)

# vacated_bed = \
# np.floor(np.multiply(x[:, state_num['h_state']], Phr)) + \
#       np.floor(np.multiply(x[:, state_num['h_state']], Phd))   
         
# a_hosp_bed = t_hosp_bed + vacated_bed.sum()
# print(a_hosp_bed)
               
# # x = np.zeros((n_age_group, 10))

# Psh = severe_prob_update(x, t_hosp_bed,state_num, n_age_group, young_age_first)

# print("Psh: ", Psh)


559.0
n_severe:  [ 10  20  25  30  50  80 100 150 300]
hosp_bed_left:  [500 490 470 445 415 365 285 185  35]
Psh:  [1.         1.         1.         1.         1.         1.
 1.         1.         0.11666667]
Psh:  [1.         1.         1.         1.         1.         1.
 1.         1.         0.11666667]


In [191]:
test = np.array(2)
print(test)

2


In [177]:
def icu_or_vent_prob_update(a_resource, r_mat_acc, n_hosp, n_critical, n_age_group, young_age_first):
    
    Phxwx = np.zeros((n_age_group))
    Phx = np.zeros((n_age_group))
    Pcx = np.zeros((n_age_group))
    used_resource = 0
    r_full_fill_mask = ""
    r_partial_fill_mask = ""
    resource_left = ""
    
    if a_resource > 0:
        n_awaiting = r_mat_acc[0]
        n_h_to_needing = r_mat_acc[1]
        n_critical_to_resource = r_mat_acc[2]
        
        zero_mask_awaiting = n_awaiting == 0
        zero_mask_h_to_needing = n_h_to_needing == 0
        zero_mask_critical_to_resource = n_critical_to_resource == 0
        
        print('n_hosp: ', n_hosp)
        print('n_hosp sum: ', n_hosp.sum())
        print('n_critical: ', n_critical)
        print('n_critical sum: ', n_critical.sum())
        print('n_awaiting: ', n_awaiting)
        print('n_awaiting sum: ', n_awaiting.sum())
        print('n_h_to_needing: ', n_h_to_needing)
        print('n_h_to_needing sum: ', n_h_to_needing.sum())
        print('n_critical_to_resource: ', n_critical_to_resource)
        print('n_critical_to_resource sum: ', n_critical_to_resource.sum())
        
        if not young_age_first:
            # If young_age_first is not applied, then let the sum of each state for all age groups to be the same
            # To do so, accumulate the sum over columns
            r_mat_acc = r_mat_acc.sum(axis=1)
            # Repeat the same number of patient over all age group for consistent probability computation
            n_awaiting = np.repeat(n_awaiting.sum(), n_age_group)
            n_hosp = np.repeat(n_hosp.sum(), n_age_group)
            n_h_to_needing = np.repeat(n_h_to_needing.sum(), n_age_group)
            n_critical = np.repeat(n_critical.sum(), n_age_group)
            n_critical_to_resource = np.repeat(n_critical_to_resource.sum(), n_age_group)
        
#         print('n_hosp: ', n_hosp)
        r_mat_acc = r_mat_acc.flatten(order='F')
        r_mat_acc = r_mat_acc.cumsum(axis=0)
        resource_left = a_resource - np.insert(r_mat_acc, 0, 0)[:-1]
        
        print("r_mat_acc: ", r_mat_acc)
        
        if young_age_first:
            r_mat_acc = np.reshape(r_mat_acc, (3, n_age_group), order='F')
            resource_left = np.reshape(resource_left, (3, n_age_group), order='F')
        else:
            r_mat_acc = np.repeat(r_mat_acc, n_age_group, axis=0).reshape(3, n_age_group)
            resource_left = np.repeat(resource_left, n_age_group, axis=0).reshape(3, n_age_group)

        r_full_fill_mask = r_mat_acc <= a_resource
        r_full_fill_mask[0, :] = np.bitwise_and(r_full_fill_mask[0, :], ~zero_mask_awaiting)
        r_full_fill_mask[1, :] = np.bitwise_and(r_full_fill_mask[1, :], ~zero_mask_h_to_needing)
        r_full_fill_mask[2, :] = np.bitwise_and(r_full_fill_mask[2, :], ~zero_mask_critical_to_resource)

        r_partial_fill_mask = np.bitwise_and(~r_full_fill_mask, resource_left >= 0)
        r_partial_fill_mask[0, :] = np.bitwise_and(r_partial_fill_mask[0, :], ~zero_mask_awaiting)
        r_partial_fill_mask[1, :] = np.bitwise_and(r_partial_fill_mask[1, :], ~zero_mask_h_to_needing)
        r_partial_fill_mask[2, :] = np.bitwise_and(r_partial_fill_mask[2, :], ~zero_mask_critical_to_resource)

#         print("r_mat_acc: ", r_mat_acc)
#         print("resource_left: ", resource_left)
#         print("r_full_fill_mask: ", r_full_fill_mask)
#         print("r_partial_fill_mask: ", r_partial_fill_mask)
        
        if r_full_fill_mask[0, :].any():
            Phxwx[r_full_fill_mask[0, :]] = 1
        if r_partial_fill_mask[0, :].any():
            Phxwx[r_partial_fill_mask[0, :]] = \
            np.array(resource_left[0, r_partial_fill_mask[0, :]]/n_awaiting[r_partial_fill_mask[0, :]]).flatten()

#         print("awaiting_resource_r_full_fill_mask: ", r_full_fill_mask[0, :])
#         print("awaiting_resource_r_partial_fill_mask: ", r_partial_fill_mask[0, :])
        print("awaiting_resource_mat_acc: ", r_mat_acc[0, :])
        print("awaiting_resource_left: ", resource_left[0, :])
        print("Phxwx: ", Phxwx)

        
#             print("n_hosp.sum(): ", n_hosp.sum())
        if r_full_fill_mask[1, :].any():            
            Phx[r_full_fill_mask[1, :]] = \
            (n_h_to_needing[r_full_fill_mask[1, :]]/n_hosp[r_full_fill_mask[1, :]]).flatten()

        if r_partial_fill_mask[1, :].any():
            print("zero_mask_h_to_needing: ", zero_mask_h_to_needing)
            print("r_partial_fill_mask[1, :]: ", r_partial_fill_mask[1, :])
            print("resource_left[1, :]:", resource_left[1, :])
            print("resource_left[1, [r_partial_fill_mask[1, :]]]:", resource_left[1, r_partial_fill_mask[1, :]])
            print("n_hosp[r_partial_fill_mask[1, :]]: ", n_hosp[r_partial_fill_mask[1, :]])
            print(np.array(resource_left[1, r_partial_fill_mask[1, :]]/n_hosp[r_partial_fill_mask[1, :]]).flatten())
            Phx[r_partial_fill_mask[1, :]] = \
            np.array(resource_left[1, r_partial_fill_mask[1, :]]/n_hosp[r_partial_fill_mask[1, :]]).flatten()
            print('test2')

#         print("h_to_resource_r_full_fill_mask: ", r_full_fill_mask[1, :])
#         print("h_to_resource_r_partial_fill_mask: ", r_partial_fill_mask[1, :])
        print("h_to_resource_mat_acc: ", r_mat_acc[1, :])
        print("h_to_resource_left: ", resource_left[1, :])
        print("Phx: ", Phx)
        
#         if n_critical.sum() > 0:
#             print("n_critical.sum(): ", n_critical.sum())
        if r_full_fill_mask[2, :].any():
            Pcx[r_full_fill_mask[2, :]] = \
            (n_critical_to_resource[r_full_fill_mask[2, :]]/n_critical[r_full_fill_mask[2, :]]).flatten()
        if r_partial_fill_mask[2, :].any():
            Pcx[r_partial_fill_mask[2, :]] = \
            np.array(resource_left[2, r_partial_fill_mask[2, :]]/n_critical[r_partial_fill_mask[2, :]]).flatten()

#         print("c_to_icu_x_r_full_fill_mask: ", r_full_fill_mask[2, :])
#         print("c_to_icu_x_r_partial_fill_mask: ", r_partial_fill_mask[2, :])
        print("c_to_icu_x_mat_acc: ", r_mat_acc[2, :])
        print("c_to_icu_x_resource_left: ", resource_left[2, :])
        print("Pcx: ", Pcx)
        
        if young_age_first:
            used_resource = np.multiply(Phxwx, n_awaiting) + \
            np.multiply(Phx, n_hosp) + np.multiply(Pcx, n_critical)
        else:
            used_resource = Phxwx[0]*n_awaiting[0] + Phx[0]*n_hosp[0] + \
            Pcx[0]*n_critical[0]
        
        used_resource = used_resource.sum()

    
    return Phxwx, Phx, Pcx, used_resource, r_full_fill_mask, r_partial_fill_mask, resource_left


In [185]:
def death_num_update(x, P_matrix, n_age_group, d_cause_num):
    
    # Death due to lack of hospital beds
    n_death_cause = len(d_cause_num)
    death_cause_mat = np.zeros((n_age_group, n_death_cause))
    
    death_cause_mat[:, d_cause_num['s']] = \
    np.floor(np.multiply(x[:, state_num['s_state']], P_matrix[state_num['s_state'],state_num['d_state'], :]))
        
    death_cause_mat[:, d_cause_num['c_hiw']] = \
    np.floor(np.multiply(x[:, state_num['c_state']], \
                    P_matrix[state_num['c_state'],state_num['d_state'], :])*(1 - icu_with_vent_rate)) + \
    np.floor(np.multiply(x[:, state_num['hiw_state']], P_matrix[state_num['hiw_state'],state_num['d_state'], :]))
        
    death_cause_mat[:, d_cause_num['c_hvw']] = \
    np.floor(np.multiply(x[:, state_num['c_state']], \
                    P_matrix[state_num['c_state'],state_num['d_state'],:])*icu_with_vent_rate) + \
    np.floor(np.multiply(x[:, state_num['hvw_state']], P_matrix[state_num['hvw_state'],state_num['d_state'],:]))
    
    death_cause_mat[:, d_cause_num['h']] = \
    np.floor(np.multiply(x[:, state_num['h_state']], P_matrix[state_num['h_state'],state_num['d_state'], :]))
    
    death_cause_mat[:, d_cause_num['i']] = \
    np.floor(np.multiply(x[:, state_num['i_state']], P_matrix[state_num['i_state'],state_num['d_state'], :]))
    
    death_cause_mat[:, d_cause_num['v']] = \
    np.floor(np.multiply(x[:, state_num['v_state']], P_matrix[state_num['v_state'],state_num['d_state'], :]))
    
    return death_cause_mat

In [1]:
# Testing icu_or_vent_prob_update

# young_age_first = True
# t_icu = 4000
# t_vent = 2000

# x = np.repeat(np.array([10, 20, 25, 30, 50, 80, 100, 150, 300]).reshape(9, 1), 10, axis=1)
# # x = np.zeros((9, 10))


# # if young_age_first:
# n_critical = x[:, state_num["c_state"]]
# n_hosp = x[:, state_num["h_state"]]
# n_icu = x[:, state_num["i_state"]]
# n_icu_vent = x[:, state_num["v_state"]]
# n_icu_awaiting = x[:, state_num["hiw_state"]]
# n_icu_vent_awaiting = x[:, state_num["hvw_state"]]
# # else:    
# #     n_critical = np.array(x.sum(axis=0)[state_num["c_state"]])
# #     n_hosp = np.array(x.sum(axis=0)[state_num["h_state"]])
# #     n_icu = np.array(x.sum(axis=0)[state_num["i_state"]])
# #     n_icu_vent = np.array(x.sum(axis=0)[state_num["v_state"]])
# #     n_icu_awaiting = np.array(x.sum(axis=0)[state_num["hiw_state"]]) # number of patients in hiw state
# #     n_icu_vent_awaiting = np.array(x.sum(axis=0)[state_num["hvw_state"]]) # number of patients in hvw state

# # Hospitalised to (ICU or ICU + vent) OR (awaiting ICU or ICU + vent) would depend on ICU and vent availability
# n_h_to_needing_icu = np.floor(np.round(n_hosp*h_i_rate)*(1 - icu_with_vent_rate))
# n_h_to_needing_icu_vent = np.round(n_hosp*h_i_rate) - n_h_to_needing_icu

# n_critical_to_icu = np.floor(n_critical*(1 - icu_with_vent_rate))
# n_critical_to_icu_vent = n_critical - n_critical_to_icu

# a_icu = t_icu - n_icu.sum() - n_icu_vent.sum() # available ICU = total - number of patients in ICU and ICU + vent
# a_vent = t_vent - n_icu_vent.sum()
# a_resource = np.amin([a_icu, a_vent])
# print("a_icu: ", a_icu)
# print("a_vent: ", a_vent)
# print("a_resource", a_resource)


# # Fill in ICU + vent first
# print("\n\nStart filling in ICU + vent\n")
# v_mat_acc = np.concatenate(([n_icu_vent_awaiting], [n_h_to_needing_icu_vent], \
#                             [n_critical_to_icu_vent]), axis=0)


# Phvwv, Phv, Pcv, used_icu_vent, _, _, _ = \
# icu_or_vent_prob_update(a_resource, v_mat_acc, n_hosp, n_critical, n_age_group, young_age_first)


# # Update available resources 
# # (by taking away resources that will be used by the number of patients who will get ICU + vent)
# # if young_age_first:
# #     used_icu_vent = np.multiply(Phvwv, n_icu_vent_awaiting) + \
# #     np.multiply(Phv, n_h_to_needing_icu_vent) + np.multiply(Pcv, n_critical_to_icu_vent)
# # else:
# #     used_icu_vent = Phvwv[0, 0]*n_icu_vent_awaiting + Phv[0, 0]*n_h_to_needing_icu_vent + \
# #     Pcv[0, 0]*n_critical_to_icu_vent
    
# a_icu = a_icu - used_icu_vent
# a_vent = a_vent - used_icu_vent

# print("used_icu_vent: ", used_icu_vent)
# print("Vent left: ", a_vent)
# print("ICU left: ", a_icu)
# print("Finish filling ICU + vent\n\n")

# # Fill in the rest of ICU
# print("Start filling in rest of ICU")
# a_resource = a_icu

# i_mat_acc = np.concatenate(([n_icu_awaiting], [n_h_to_needing_icu], \
#                             [n_critical_to_icu]), axis=0)

# Phvwi, Phi, Pci, used_icu, _, _, _ = \
# icu_or_vent_prob_update(a_resource, i_mat_acc, n_hosp, n_critical, n_age_group, young_age_first)

# # if young_age_first:
# #     used_icu = np.multiply(Phvwi, n_icu_awaiting) + \
# #     np.multiply(Phi, n_h_to_needing_icu) + np.multiply(Pci, n_critical_to_icu)
# # else:
# #     used_icu = Phvwi[0, 0]*n_icu_awaiting + Phi[0, 0]*n_h_to_needing_icu + \
# #     Pci[0, 0]*n_critical_to_icu
    
# a_icu = a_icu - used_icu

# print("used_icu: ", used_icu)
# print("ICU left: ", a_icu)
# print("Finish filling ICU")

In [2]:
def update_prob(P_matrix, x, t_hosp_bed, t_icu, t_vent, state_num, n_age_group=9, young_age_first=True):
    # If young_age first, then tyounger people have higher priority to get medical resouces 
    # (unfortunate in this situation)
    
    # Update the number of empty hospital beds and the probability of Psh
    # Patients who will recover or pass away will vacate the hospital bed
    vacated_bed = np.floor(np.multiply(x[:, state_num['h_state']], Phr)) + \
    np.floor(np.multiply(x[:, state_num['h_state']], Phd))
    
    a_hosp_bed = t_hosp_bed - x.sum(axis=0)[state_num["h_state"]] + vacated_bed.sum()
    
    Psh = severe_prob_update(x, a_hosp_bed, state_num, n_age_group, young_age_first)
        
    # There are 3 pathways of getting into ICU or ICU + ventilator
    # 1. Patients whose conditions have already worsened (in hiw and hvw state) and are waiting for ICU or ICU + ventilator
    # 2. Hospitalised patients whose conditions would (in h state) worsen and need ICU or ICU + ventilator
    # 3. Patients who have not been hospitalised but present critical conditions
    # To simplify the model, the order in which these patients would been filled would follow 1, 2 and 3.
    # Note that for patients in 2, if they don't get ICU or ICU + ventilator, they would all go to 1 (hiw and hvw)
    # Update critical case probability
    # Fill in ventilators first
    n_severe = x[:, state_num["s_state"]]
    n_critical = x[:, state_num["c_state"]]
    n_hosp = x[:, state_num["h_state"]]
    n_icu = x[:, state_num["i_state"]]
    n_icu_vent = x[:, state_num["v_state"]]
    n_icu_awaiting = x[:, state_num["hiw_state"]]
    n_icu_vent_awaiting = x[:, state_num["hvw_state"]]
    
    
    # Hospitalised to (ICU or ICU + vent) OR (awaiting ICU or ICU + vent) would depend on ICU and vent availability
    n_h_to_needing_icu = np.floor(np.round(n_hosp*h_i_rate)*(1 - icu_with_vent_rate))
    print('n_h_to_needing_icu: ', n_h_to_needing_icu)
    n_h_to_needing_icu_vent = np.round(n_hosp*h_i_rate) - n_h_to_needing_icu
    print('n_h_to_needing_icu_vent: ', n_h_to_needing_icu_vent)

    n_critical_to_icu = np.floor(n_critical*(1 - icu_with_vent_rate))
    n_critical_to_icu_vent = n_critical - n_critical_to_icu
    
    vacated_icu = np.floor(np.multiply(x[:, state_num['i_state']], Pir)) + \
    np.floor(np.multiply(x[:, state_num['i_state']], Pid)) + \
    np.floor(np.multiply(x[:, state_num['v_state']], Pvr)) + \
    np.floor(np.multiply(x[:, state_num['v_state']], Pvd))
    
    vacated_vent = np.floor(np.multiply(x[:, state_num['v_state']], Pvr)) + \
    np.floor(np.multiply(x[:, state_num['v_state']], Pvd))

    # available ICU = total - number of patients in ICU and ICU + vent
    a_icu = t_icu - n_icu.sum() - n_icu_vent.sum() + vacated_icu.sum()
    a_vent = t_vent - n_icu_vent.sum() + vacated_vent.sum()
    a_resource = np.amin([a_icu, a_vent])
#     print("a_icu: ", a_icu)
#     print("a_vent: ", a_vent)
#     print("a_resource", a_resource)

    # Fill in ICU + vent first
    v_mat_acc = np.concatenate(([n_icu_vent_awaiting], [n_h_to_needing_icu_vent], \
                                [n_critical_to_icu_vent]), axis=0)


    Phvwv, Phv, Pcv, used_icu_vent, _, _, _ = \
    icu_or_vent_prob_update(a_resource, v_mat_acc, n_hosp, n_critical, n_age_group, young_age_first)


    # Update available resources 
    # (by taking away resources that will be used by the number of patients who will get ICU + vent)
    a_icu = a_icu - used_icu_vent
    a_vent = a_vent - used_icu_vent
    
    
    # Fill in the rest of ICU beds
    i_mat_acc = np.concatenate(([n_icu_awaiting], [n_h_to_needing_icu], \
                                [n_critical_to_icu]), axis=0)

    Phiwi, Phi, Pci, used_icu, _, _, _ = \
    icu_or_vent_prob_update(a_icu, i_mat_acc, n_hosp, n_critical, n_age_group, young_age_first)

    # Update available resources 
    a_icu = a_icu - used_icu
    
    
    # Update probabilities and the P_matrix
    
    # Severe cases
    # The porbability of severe cases that will develop into critical state if not hospitalised is adjusted based on 
    # hospitalised (left severe cases untreated = (1 - Psh))
    severe_zero_mask = n_severe == 0
    print("severe_zero_mask: ", severe_zero_mask)
    nPsc = np.zeros((n_age_group))
    nPsd = np.zeros((n_age_group))
    if not severe_zero_mask.all():
        nPsc[~severe_zero_mask] = (1 - Psh[~severe_zero_mask])*Psc[~severe_zero_mask]
    # The probability of severe cases that will pass away if not hospitalised is adjusted based on 
    # hospitalised (left severe cases untreated = (1 - Psh))
        nPsd[~severe_zero_mask] = (1 - Psh[~severe_zero_mask])*Psd[~severe_zero_mask]
    
    Pss = 1 - nPsc - Psh - nPsd
    print("Psh: ", Psh)
    print("Pss: ", Pss)
    
    # Critical cases
    # The probability of critical cases that will pass away if not sent into ICU is adjusted based on 
    # ICU and ICU + vent (left critical cases untreated = (1 - Pci - Pcv))
    critical_zero_mask = n_critical == 0
    nPcd = np.zeros((n_age_group))
    if not critical_zero_mask.all():
        nPcd[~critical_zero_mask] = (1 - Pci[~critical_zero_mask] - Pcv[~critical_zero_mask])*Pcd[~critical_zero_mask]
    nPcd[np.absolute(nPcd) < 1e-6] = 0
    Pcc = 1 - Pci - Pcv - nPcd
    print("nPcd: ", nPcd)
    print("Pcc: ", Pcc)
    
    
    # Hospitalised cases
    # The probability of hospitalised cases that will recover or pass away is adjusted based on 
    # hospitalised to other states (left hospitalised cases = (1 - Phi - Phv - Phhiw - Phhvw))
    h_zero_mask = n_hosp == 0
    Phhiw = np.zeros((n_age_group))
    Phhvw = np.zeros((n_age_group))
    if not h_zero_mask.all():
        if a_icu == 0:
            Phhiw[~h_zero_mask] = h_i_rate*(1 - icu_with_vent_rate) - Phi[~h_zero_mask]
        if a_vent == 0:
            Phhvw[~h_zero_mask] = h_i_rate*icu_with_vent_rate - Phv[~h_zero_mask]
#   # sometimes due to rounding, Phi can go over the limit (h_i_rate*(1 - icu_with_vent_rate)) a little
    Phhiw[Phhiw < 0] = 0
    
#   # sometimes due to rounding, Phv can go over the limit (h_i_rate*(1 - icu_with_vent_rate)) a little
    Phhvw[Phhvw < 0] = 0
    
    nPhr = (1 - Phi - Phv - Phhiw - Phhvw)*Phr
    nPhd = (1 - Phi - Phv - Phhiw - Phhvw)*Phd
    Phh = 1 - Phi - Phv - Phhiw - Phhvw - nPhd - nPhr
    print("Phi: ", Phi)
    print("Phhiw: ", Phhiw)
    print("Phv: ", Phv)
    print("Phhvw: ", Phhvw)
    print("nPhr: ", nPhr)
    print("nPhd: ", nPhd)
    print("Phh: ", Phh)    
    
    # The probability of hospitalised cases awaiting ICU will pass away is adjusted based on 
    # hospitalised cases awaiting ICU to other states 
    # (left hospitalised cases awaiting ICU = (1 - Phiwi))
    hiw_zero_mask = n_icu_awaiting == 0
    nPhiwd = np.zeros((n_age_group))
    if not hiw_zero_mask.all():
        nPhiwd[~hiw_zero_mask] = (1 - Phiwi[~hiw_zero_mask])*Phiwd[~hiw_zero_mask]
    Phiwhiw = 1 - Phiwi - nPhiwd
    print("nPhiwd: ", nPhiwd)
    print("Phiwhiw: ", Phiwhiw)
    
    # The probability of hospitalised cases awaiting ICU + vent will pass away is adjusted based on 
    # hospitalised cases awaiting ICU + vent to other states 
    # (left hospitalised cases awaiting ICU + vent = (1 - Phvwv))
    hvw_zero_mask = n_icu_vent_awaiting == 0
    nPhvwd = np.zeros((n_age_group))
    if not hvw_zero_mask.all():
        nPhvwd[~hvw_zero_mask] = (1 - Phvwv[~hvw_zero_mask])*Phvwd[~hvw_zero_mask]
    Phvwhvw = 1 - Phvwv - nPhvwd
    print("nPhvwd: ", nPhvwd)
    print("Phvwhvw: ", Phvwhvw)
    
    P_matrix[state_num["s_state"], state_num["h_state"], :] = Psh
    P_matrix[state_num["s_state"], state_num["c_state"], :] = nPsc
    P_matrix[state_num["s_state"], state_num["d_state"], :] = nPsd
    P_matrix[state_num["s_state"], state_num["s_state"], :] = Pss
    
    P_matrix[state_num["c_state"], state_num["i_state"], :] = Pci
    P_matrix[state_num["c_state"], state_num["v_state"], :] = Pcv
    P_matrix[state_num["c_state"], state_num["d_state"], :] = nPcd
    P_matrix[state_num["c_state"], state_num["c_state"], :] = Pcc
    
    # Update Ph
    # Update Phi (hospitalised to critical)
    P_matrix[state_num["h_state"], state_num["i_state"], :] = Phi
    # Update Phv (hospitalised to critical + ventilator)
    P_matrix[state_num["h_state"], state_num["v_state"], :] = Phv
    # Update Phhiw (hospitalised to critical but awaiting)
    P_matrix[state_num["h_state"], state_num["hiw_state"], :] = Phhiw    
    # Update Phhvw (hospitalised to critical + ventilator but awaiting)
    P_matrix[state_num["h_state"], state_num["hvw_state"], :] = Phhvw
    P_matrix[state_num["h_state"], state_num["r_state"], :] = nPhr
    P_matrix[state_num["h_state"], state_num["d_state"], :] = nPhd
    # Update Phh
    P_matrix[state_num["h_state"], state_num["h_state"], :] = Phh
    
    # Update Phiwi (awaiting ICU to ICU)
    P_matrix[state_num["hiw_state"], state_num["i_state"], :] = Phiwi
    P_matrix[state_num["hiw_state"], state_num["d_state"], :] = nPhiwd
    P_matrix[state_num["hiw_state"], state_num["hiw_state"], :] = Phiwhiw
    
    # Update Phvwv (awaiting ICU + ventilator to ICU + ventilator)
    P_matrix[state_num["hvw_state"], state_num["v_state"], :] = Phvwv
    P_matrix[state_num["hvw_state"], state_num["d_state"], :] = nPhvwd
    P_matrix[state_num["hvw_state"], state_num["hvw_state"], :] = Phvwhvw
    
    
#     print("P_matrix: ", P_matrix)
    
#     np.ones((1, n_age_group))*h_i_rate/avg_h_to_i * h_to_i_control
    
    
    
    
    return P_matrix, a_hosp_bed, a_icu, a_vent

In [3]:
# Test update_prod() function
# young_age_first = False
# t_icu = 2700
# t_vent = 1080
# t_hosp_bed = 2000

# y = np.zeros((n_age_group, n_state))

# x = np.repeat(np.array([10, 20, 25, 30, 50, 80, 100, 150, 300]).reshape(9, 1), 10, axis=1)

# P_mat, _, _, _ = update_prob(P_matrix, x, t_hosp_bed, t_icu, t_vent, state_num)

# for i in range(n_age_group):
#     y[i, :] = np.dot(x[i, :], P_mat[:,:,i])

# # print("P_mat: ", P_mat)
# print(np.argwhere(P_mat < 0))
# print("x: ", x)
# print("y: ", y)

In [182]:
# For loop method: inefficient, incomplete


# if a_vent <= a_icu: # If the number of ventilators being the limiting factor
#             if icu_vent_list[n] <= a_vent:
#                 # critical case has the probability of (icu_with_vent_rate) going to ICU + vent 
#                 # and (1 - icu_with_vent_rate) going to ICU
#                 P[n] = 1
#                 a_vent = icu_vent_list[n]
#             else:
#                 Pv = a_vent/(n_h_to_icu_vent_total + n_critical_total*icu_with_vent_rate)
#                 Pcv = np.ones((1, n_age_group))*Pv
#                 Phvwv = np.ones((1, n_age_group))*Pv
#                 a_vent = 0
#             a_icu = a_icu - a_vent
#         else: # else the number of ICU being the limiting factor
#             if (n_h_to_icu_vent_total + n_critical_total*icu_with_vent_rate) <= a_icu:
#                 # critical case has the probability of (icu_with_vent_rate) going to ICU + vent 
#                 # and (1 - icu_with_vent_rate) going to ICU
#                 Pcv = np.ones((1, n_age_group))*icu_with_vent_rate 
#                 Phvwv = np.ones((1, n_age_group))
#                 a_icu = a_icu - n_h_to_icu_vent_total - n_critical_total*icu_with_vent_rate

#             else:
#                 Pv = a_icu/(n_h_to_icu_vent_total + n_critical_total*icu_with_vent_rate)
#                 Pcv = np.ones((1, n_age_group))*Pv
#                 Phvwv = np.ones((1, n_age_group))*Pv
#                 a_icu = 0
#             a_vent = a_vent - a_icu
    
#     # Fill in the ICUs
#     if (n_h_to_icu_total + n_critical_total*(1 - icu_with_vent_rate)) <= a_icu:
#         # critical case has the probability of (icu_with_vent_rate) going to ICU + vent 
#         # and (1 - icu_with_vent_rate) going to ICU
#         Pci = np.ones((1, n_age_group))*icu_with_vent_rate 
#         Phiwi = np.ones((1, n_age_group))
#         a_icu = a_icu - n_h_to_icu_total - n_critical_total*(1 - icu_with_vent_rate)
            
#     else:
#         Pi = a_icu/(n_h_to_icu_vent_total + n_critical_total*(1 - icu_with_vent_rate))
#         Pci = np.ones((1, n_age_group))*Pv
#         Phiwi = np.ones((1, n_age_group))*Pv
#         a_icu = 0

In [ ]:
# Probability update
# Phvwv = np.zeros((1, n_age_group))
# # Pcv = np.zeros((1, n_age_group))

# # Fill in Phvwv
# if v_full_fill_mask[0, :].any():
#     Phvwv[:, v_full_fill_mask[0, :]] = 1
# if v_partial_fill_mask[0, :].any():
#     Phvwv[:, v_partial_fill_mask[0, :]] = \
#     np.array(v_left[0, :]/n_icu_vent_awaiting)[v_partial_fill_mask[0, :]]

# print("v_full_fill_mask_awaiting_icu_vent: ", v_full_fill_mask[0, :])
# print("v_partial_fill_mask_awaiting_icu_vent: ", v_partial_fill_mask[0, :])
# print("Phvwv: ", Phvwv)

# Phv = np.zeros((1, n_age_group))

# if v_full_fill_mask[1, :].any():
#     Phv[:, v_full_fill_mask[1, :]] = 1
# if v_partial_fill_mask[1, :].any():
#     Phv[:, v_partial_fill_mask[1, :]] = \
#     np.array(v_left[1, :]/n_h_to_needing_icu_vent)[v_partial_fill_mask[1, :]]

# print("v_full_fill_mask_h_to_icu_vent: ", v_full_fill_mask[1, :])
# print("v_partial_fill_mask_h_to_icu_vent: ", v_partial_fill_mask[1, :])
# print("Phv: ", Phv)

# Pcv = np.zeros((1, n_age_group))

# if v_full_fill_mask[2, :].any():
#     Pcv[:, v_full_fill_mask[2, :]] = 1
# if v_partial_fill_mask[2, :].any():
#     Pcv[:, v_partial_fill_mask[2, :]] = \
#     np.array(v_left[2, :]/n_critical_to_icu_vent)[v_partial_fill_mask[2, :]]

# print("v_full_fill_mask_c_to_icu_vent: ", v_full_fill_mask[2, :])
# print("v_partial_fill_mask_c_to_icu_vent: ", v_partial_fill_mask[2, :])
# print("Pcv: ", Pcv)

In [ ]:
# def update_icu_and_vent(t_icu, t_vent, n_icu_awaiting, n_icu_vent_awaiting,n_h_to_needing_icu, n_h_to_needing_icu_vent, n_critical, icu_with_vent_rate, young_age_first=True):
    
#     if young_age_first:
#         n_critical = x[:, c_state_num]
#         n_hosp = x[:, h_state_num]
#         n_icu = x[:, i_state_num]
#         n_icu_vent = x[:, v_state_num]
#         n_icu_awaiting = x[:, hiw_state_num]
#         n_icu_vent_awaiting = x[:, hvw_state_num]
#     else:    
#         n_critical = np.array(x.sum(axis=0)[c_state_num])
#         n_hosp = np.array(x.sum(axis=0)[h_state_num])
#         n_icu = np.array(x.sum(axis=0)[i_state_num])
#         n_icu_vent = np.array(x.sum(axis=0)[v_state_num])
#         n_icu_awaiting = np.array(x.sum(axis=0)[hiw_state_num]) # number of patients in hiw state
#         n_icu_vent_awaiting = np.array(x.sum(axis=0)[hvw_state_num]) # number of patients in hvw state
        
#     # Hospitalised to (ICU or ICU + vent) OR (awaiting ICU or ICU + vent) would depend on ICU and vent availability
#     n_h_to_needing_icu = n_hosp*h_i_rate*(1 - icu_with_vent_rate)
#     n_h_to_needing_icu_vent = n_hosp*h_i_rate*icu_with_vent_rate
    
    
#     a_icu = t_icu - n_icu_total.sum() - n_icu_vent.sum() # available ICU = total - number of patients in ICU and ICU + vent
#     a_vent = t_vent - n_icu_vent.sum()
        
#     vent_mat_acc = np.concatenate(([n_icu_vent_awaiting], [n_h_to_needing_icu_vent], \
#                                    [n_critical]), axis=0).flatten(order='F')
#     vent_mat_acc = vent_mat_acc.cumsum(axis=0)
#     vent_left = a_vent - np.insert(vent_mat_acc, 0, 0)[:-1]
    
#     if len(n_critical) == 1:
#         vent_mat_acc = np.repeat(vent_mat_acc, n_age_group, axis=0)
#         vent_left = np.repeat(vent_left, n_age_group, axis=0)
    
#     vent_full_fill_mask = vent_mat_acc <= a_vent
    
#     vent_partial_fill_mask = np.bitwise_and(~vent_full_fill_mask, vent_left >= 0)


#     Phvwv = np.zeros((1, n_age_group))
#     Phv = np.zero((1, n_age_group))
#     Pcv = np.zero((1, n_age_group))
    
#     # Fill in Phvwv
#     if vent_full_fill_mask[0:n_age_group].any():
#         Phvwv[:, vent_full_fill_mask[0:n_age_group]] = 1
#     if vent_partial_fill_mask[0:n_age_group].any():
#         Phvwv[:, partial_fill_mask] = np.array(vent_left/n_icu_vent_awaiting)[vent_partial_fill_mask[0:n_age_group]]
        
# #     a_hosp_bed = a_hosp_bed - np.dot(Psh, n_severe)


# #     Psh = np.zeros((n_severe.shape[0],1))

# #     Psh[full_fill_mask] = 1
# #     if partial_fill_mask.any():
# #         Psh[partial_fill_mask, :] = np.array(hosp_bed_left/n_severe)[partial_fill_mask]
        
    
    
    
    
    
    
    
#     icu_vent_list = [n_icu_vent_awaiting, n_h_to_needing_icu_vent, n_critical*icu_with_vent_rate]
#     Pv = [0, 0, 0]
#     Pi = [0, 0, 0]
    
#     for n in range(3):
#         if a_vent <= a_icu: # If the number of ventilators being the limiting factor
#             if icu_vent_list[n] <= a_vent:
#                 # critical case has the probability of (icu_with_vent_rate) going to ICU + vent 
#                 # and (1 - icu_with_vent_rate) going to ICU
#                 Pv[n] = 1
#                 a_vent = a_vent - icu_vent_list[n]
#                 a_icu = a_icu - icu_vent_list[n]
#             else:
#                 Pv[n] = a_vent/icu_vent_list[n]
#                 a_vent = 0
#                 if icu_vent_list[n] <= a_icu:
#                     a_icu = a_icu - icu_vent_list[n]
#                 else:
#                     a_icu = 0
#         else: # else the number of ICU being the limiting factor
#             if icu_vent_list[n] <= a_icu:
#                 # critical case has the probability of (icu_with_vent_rate) going to ICU + vent 
#                 # and (1 - icu_with_vent_rate) going to ICU
#                 Pv[n] = 1
#                 a_icu = a_icu - icu_vent_list[n]
#                 a_vent = a_vent - icu_vent_list[n]

#             else:
#                 Pv = a_icu/icu_vent_list[n]
#                 a_icu = 0
#                 if icu_vent_list[n] <= a_vent:
#                     a_vent = a_vent - icu_vent_list[n]
#                 else:
#                     a_vent = 0
    
#     # Fill in the ICUs
#     if (n_h_to_icu_total + n_critical_total*(1 - icu_with_vent_rate)) <= a_icu:
#         # critical case has the probability of (icu_with_vent_rate) going to ICU + vent 
#         # and (1 - icu_with_vent_rate) going to ICU
#         Pci = np.ones((1, n_age_group))*icu_with_vent_rate 
#         Phiwi = np.ones((1, n_age_group))
#         a_icu = a_icu - n_h_to_icu_total - n_critical_total*(1 - icu_with_vent_rate)
            
#     else:
#         Pi = a_icu/(n_h_to_icu_vent_total + n_critical_total*(1 - icu_with_vent_rate))
#         Pci = np.ones((1, n_age_group))*Pv
#         Phiwi = np.ones((1, n_age_group))*Pv
#         a_icu = 0